## Webscraping with Beautiful Soup

Import libraries for webscraping

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

Load website and read into soup object

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

website_url = requests.get(url).text

soup = BeautifulSoup(website_url,'lxml')
#print(soup.prettify())

Next we identify the table inside the soup: it is marked as class _'wikitable sortable'_

In [3]:
table = soup.find('table',{'class':'wikitable sortable'})
table

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>
<tr>
<td>M6A</td>

We loop over the rows of the table **tr**. 

The header (**th**) is read into a list of _Cols_

Table rows (**tr**) are read into a lists. We know that we expect 3 columns from the website, but we could also test for the number of columns first. We are also stripping the text of any special characters such as '\n', which messes with later processing of the data frame.

In [4]:
PostCodes = []
Boroughs = []
Hoods = []

for row in table.findAll("tr"):
    data = row.findAll("td")
    if len(data) ==3:
        txt = data[0].findAll(text=True)
        PostCodes.append(txt[0].strip())
        txt = data[1].findAll(text=True) 
        Boroughs.append(txt[0].strip())
        txt = data[2].findAll(text=True) 
        Hoods.append(txt[0].strip())
    else:
        Cols = []
        data = row.findAll("th")
        for i in data:
            txt = i.findAll(text=True)
            txt = txt[0].strip()
            Cols.append(txt)

## Pandas processing of data frame

We read the lists into a pandas **df**

In [5]:
df = pd.DataFrame(list(zip(PostCodes, Boroughs, Hoods)), 
               columns =Cols) 
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


Now, extracting all rows that have a valid _Borough_.

In [6]:
# drop rows with unassigned Boroughs
df=df[df['Borough'] != 'Not assigned']
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


I am now selecting and reassigning all rows, where _Neighbourhood_ is not assigned with the corresponding value for _Borough_.

In [7]:
df['Neighbourhood'][df['Neighbourhood']=='Not assigned'] =df['Borough'][df['Neighbourhood']=='Not assigned']
df.head(10) 

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


## Making the final table

I first create a set of unique postcodes, which I then iterate through to get the corresponding _Neighborhoods_, which are saved as a list. The list is converted to a comma-separated string. 

I then extract the _Borough_ for the postcode and append all this to a new dataframe. 

In [8]:
# get the unique bourous
Codes = set(df['Postcode'])

# loop over Borough and get list of neighborhoods

df_grouped = pd.DataFrame(columns=Cols)

for code in Codes:
    n = list(df['Neighbourhood'][df['Postcode']==code])
    
    s = ', '.join(n) 
    b = list(set(df['Borough'][df['Postcode']==code]))
    
    df_grouped = df_grouped.append({'Postcode':code , 'Borough': b[0] , 'Neighbourhood' :s},ignore_index=True)

df_grouped.head(100)

,Postcode,Borough,Neighbourhood
0,M5N,Central Toronto,Roselawn
1,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel"
2,M6G,Downtown Toronto,Christie
3,M6J,West Toronto,"Little Portugal, Trinity"
4,M9M,North York,"Emery, Humberlea"
5,M1W,Scarborough,L'Amoreaux West
6,M3K,North York,"CFB Toronto, Downsview East"
7,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo..."
8,M2J,North York,"Fairview, Henry Farm, Oriole"
9,M5C,Downtown Toronto,St. James Town


## Reporting the shape of the DF

In [9]:
df_grouped.shape

(103, 3)

## Save dataframe to csv for next excercise

In [10]:
df_grouped.to_csv('Toronto_PostCodes.csv')